In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import param
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

In [7]:
# Enable Panel extensions
pn.extension()

In [12]:
class ScatterplotApp():
    """Interactive scatterplot application using Panel"""
    
    def __init__(self, df):
        self.df = df
        self.numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Create dropdown widgets for x and y variables
        self.x_select = pn.widgets.Select(
            name="X Variable", 
            value=self.numeric_columns[0] if self.numeric_columns else None,
            options=self.numeric_columns
        )
        
        self.y_select = pn.widgets.Select(
            name="Y Variable",
            value=self.numeric_columns[1] if len(self.numeric_columns) > 1 else self.numeric_columns[0],
            options=self.numeric_columns
        )
        
        # Create the plot pane
        self.plot_pane = pn.pane.Bokeh(self.create_plot(), sizing_mode='stretch_width', height=500)
        
        # Watch for changes in the dropdowns
        self.x_select.param.watch(self.update_plot, 'value')
        self.y_select.param.watch(self.update_plot, 'value')
    
    def create_plot(self):
        """Create the bokeh scatterplot"""
        x_col = self.x_select.value
        y_col = self.y_select.value
        
        if x_col is None or y_col is None:
            # Return empty plot if no columns selected
            p = figure(width=600, height=400, title="Select variables to plot")
            return p
        
        # Create the scatter plot
        p = figure(
            width=600, 
            height=400,
            title=f"{y_col} vs {x_col}",
            x_axis_label=x_col,
            y_axis_label=y_col,
            tools="pan,wheel_zoom,box_zoom,reset,save"
        )
        
        # Filter out any NaN values for plotting
        plot_df = self.df[[x_col, y_col]].dropna()
        
        # Add scatter points
        p.scatter(
            plot_df[x_col], 
            plot_df[y_col], 
            size=8, 
            alpha=0.6,
            color='steelblue'
        )
        
        return p
    
    def update_plot(self, event):
        """Update the plot when dropdown selections change"""
        self.plot_pane.object = self.create_plot()
    
    def get_app(self):
        """Return the Panel app layout"""
        return pn.Column(
            "# Interactive Scatterplot",
            pn.Row(self.x_select, self.y_select),
            self.plot_pane,
            sizing_mode='stretch_width'
        )

def create_scatterplot_app(df):
    """
    Create and serve a Panel app for interactive scatterplotting
    
    Parameters:
    df (pd.DataFrame): The pandas DataFrame to plot
    
    Returns:
    Panel app object that can be served
    """
    if df.empty:
        return pn.pane.Markdown("## No data provided")
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) == 0:
        return pn.pane.Markdown("## No numeric columns found in DataFrame")
    app = ScatterplotApp(df)
    return app.get_app()

In [13]:
np.random.seed(42)
sample_df = pd.DataFrame({
    'height': np.random.normal(170, 10, 100),
    'weight': np.random.normal(70, 15, 100),
    'age': np.random.randint(18, 80, 100),
    'income': np.random.lognormal(10, 1, 100),
    'category': np.random.choice(['A', 'B', 'C'], 100)
})

# Create and serve the app
app = create_scatterplot_app(sample_df)

In [15]:
app

Column(sizing_mode='stretch_width')
    [0] Markdown(str)
    [1] Row
        [0] Select(name='X Variable', options=['height', 'weight', ...], value='height')
        [1] Select(name='Y Variable', options=['height', 'weight', ...], value='weight')
    [2] Bokeh(figure, height=500, sizing_mode='stretch_width')